** MACHINE LEARNING NO SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DA WEBCAM - OpenVC, MTCNN, LabelEncoder **

In [ ]:
import os
import subprocess
from cv2 import *
import cv2
from mtcnn.mtcnn import MTCNN

# Classe para regitrar/gravar a imagem em seu computador
class RegisterImg:
    def __init__(self):
        # Local Gravar Imagem Do Participante
        self.grv_img = "C:\\zproject-course\\deeplearn\\dataset\\"
        # Carregar Algoritmo Para Detectar Forma e Formato do rosto
        self.detector = MTCNN()
        print("= MACHINE LEARNING NO SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder =")

    # Detectar Face Pela Webcam E Retornar Coordenadas Do Local Da Face
    def face_mtcnn_extractor(self, frame):
        result = self.detector.detect_faces(frame)
        return result
    
    # Pegar As Coordenadas Da Face E As Retornar Em Determinadas Variáveis
    def face_localizer(self, person):
        bounding_box = person['box']
        x1, y1 = abs(bounding_box[0]), abs(bounding_box[1])
        width, height = bounding_box[2], bounding_box[3]
        x2, y2 = x1 + width, y1 + height
        return x1, y1, x2, y2, width, height

    # Chamar Methodo para Gravar A Imagem Visualizada Atraves da Webcam
    def saveperson(self):
        os.system('cls')
        print("Gravando...")
        print("Digitar Dados Do Participante")
        matricula = 0
        try:
            matricula = int(input("Número Matrícula -> "))
        except ValueError:
            print("")
            print("== Atenção ==")
            print("Digite Valor Númerico")
            print("              =======")
            menu = MainMenu()
            menu.homemenu()

        nome = input("Nome Completo -> ")
        
        # Inicializar Objeto Que Habilita Camera/WebCam Para Capturar Imagem. 0 = WebCam Notebook/PC
        cam = cv2.VideoCapture(0)

        # Enquanto A Webcam Estiver Ativa -Visualizando Imagem
        while True:
            # Detectar Face Em Cada Frame
            ret, frame = cam.read()
            if not ret:
                break
            # Detectar Face Pela Webcam E Retornar Coordenadas Do Local Da Face
            result = self.face_mtcnn_extractor(frame)
            # Exibir A Janela Ate Uma Tecla Ser Pressionada Para Uma Ação Qualquer
            k = cv2.waitKey(1)
            # Desenhar Retângulo Na Face Detectada Utilizando As Coordenadas
            for person in result:
                # Trazar As Coordenadas Da Face E As Colocar Em Determinadas Variáveis
                x1, y1, x2, y2, width, height = self.face_localizer(person)
                # Desenhar Linhas Do Retangulo Sobre A Imagem
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 155, 255), 2)
            # Mostrar Janela-Frame Com Descrição No Topo
            cv2.imshow("Coletar Imagens - ENTER = Gravar / ESC = Finalizar", frame)
            # Verificar Se Foi Pressionado A Tecla 27=ESC Para Fechar Webcam
            if k%256 == 27:
                k = cv2.waitKey(1)
                print("")
                print("ESC pressionado. Fechando Câmera")
                break
                # Verificar Se Foi Pressionado A Tecla 13=ENTER Para Gravar Imagem Através Da Webcam
            elif k%256 == 13: # Se pressionar o ENTER
                # Converter Imagem À Escala Cinza
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                # Criar Diretótio Com Nome do Participante
                os.mkdir(self.grv_img+"train\\"+nome+"_"+str(matricula))
                os.mkdir(self.grv_img+"test\\"+nome+"_"+str(matricula))
                # Gravar Imagem Com Nome Filmado + Numero De Matrícula
                cv2.imwrite(self.grv_img+"train\\"+nome+"_"+str(matricula)+"\\part-"+str(matricula)+".jpg", gray[y1:y2,x1:x2])
                cv2.imwrite(self.grv_img+"test\\"+nome+"_"+str(matricula)+"\\part-"+str(matricula)+".jpg", gray[y1:y2,x1:x2])
                print("Imagem gravada!")
        # Ao Final Da Uso da Webcam Limpa As Variáveis Objeto Webcam E
        # Janelas Utilizadas Neste Processo, Respectivamente
        cam.release()
        cv2.destroyAllWindows()
        return

In [ ]:
import datetime
import time
from os import listdir
from os.path import isdir
from PIL import Image
from numpy import savez_compressed, asarray, load, expand_dims
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle

# Classe Que Treina O Modelo svm-SVC Para Reconhecer A Imagem 
class FaceTrainer:
    def __init__(self):
        # Local Armazenar Imagens De Treino E Teste
        self.datasetpath = "C:\\zproject-course\\deeplearn\\dataset\\"
        # Local Armazenar Imagens De Treino E Teste Compactadas
        self.faces_npz = "C:\\zproject-course\\deeplearn\\dataset\\faces_dataset.npz"
        # Local Armazenar Algoritimo De Reconhecimento Facial Treinado Com Centenas de Milhares de Imagens
        self.keras_facenet = "C:\\zproject-course\\deeplearn\\facenet\\facenet_keras.h5"
        # Local Armazenar 
        self.faces_embeddings = "C:\\zproject-course\\deeplearn\\facerecognition\\faces_dataset_embeddings.npz"
        self.svm_classifier = "C:\\zproject-course\\deeplearn\\facerecognition\\SVM_classifier.sav"
        return

    # Retornar Somente Face Das Imagens De Treino e Teste Convertidas Em Array
    def load_dataset(self, directory):
        X = []
        y = []
        # Trazer Somente A Face De Cada Imagem Nos Diretórios Informados
        for subdir in listdir(directory):
            path = directory + subdir + '\\'
            # Desconsiderar Qualquer Arquivo Que Pode Estar No Diretorio
            if not isdir(path):
                continue
            # Carregar Todas As Faces No Subdiretório
            faces = self.load_faces(path)
            # Criar Legenda/Etiqueta
            labels = [subdir for _ in range(len(faces))]
            print("carregado {} exemplos para classificar: {}".format(len(faces), subdir))
            X.extend(faces)
            y.extend(labels)
        return asarray(X), asarray(y)

    # Retornar Apenas A Face De Cada Imagem Gravada Em Determinado Dataset, Com Dimensão 160x160
    def load_faces(self, directory):
        faces = []
        # Percorrer Por Todos Os Diretórios
        for filename in listdir(directory):
            path = directory + filename
            # Retorna A Face De Cada Imagem Gravada Em Determinado Dataset Com Dimensão 160x160
            face = self.extract_face(path)
            faces.append(face)
        return faces

    # Extrair A Face De Cada Imagem Gravada Em Determinado Dataset
    def extract_face(self, filename, required_size=(160, 160)):
        image = Image.open(filename)
        image = image.convert('RGB')
        pixels = asarray(image)
        # Instanciar Objeto Para Detectar Face Da Imagem Convertida Em Array
        detector = MTCNN()
        results = detector.detect_faces(pixels)
        # Extrair Os Limites Da Caixa Da Primeira Face
        x1, y1, width, height = results[0]['box']
        # Convertendo Coordenadas x, y Em Valores Absolutos (Corrigir Inconsistencia)
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # Extrair Apenas A Face
        face = pixels[y1:y2, x1:x2]
        # Redimensionar Pixels Ao Tamanho Do Modelo
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array

    # Criar Arquivo Comprimido De Todas As Faces Convertidas Em Array, Que Serão Treinadas E Validadas
    def create_faces_npz(self):
        trainX, trainy = self.load_dataset(self.datasetpath+"train\\")
        print("Treinando conjunto de dados . . .")
        testX, testy = self.load_dataset(self.datasetpath+"test\\")
        print("Testando conjunto de dados . . .")
        # Comprimir As Imagens Convertidas em Array E Gravar Em Determinado Local
        savez_compressed(self.faces_npz, trainX, trainy, testX, testy)
        return

    # Retornar O Arquivo Criado Com A Predição Das Imagens De Treino E Teste Embedded
    def create_faces_embedding_npz(self):
        # Traz O Dataset Com As Faces Em Array E Exibe Informação Em Tela
        data = load(self.faces_npz)
        trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
        print('Carregado: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
        # Carrega O Modelo Facenet facenet_keras.h5 E Exibe Informação em Tela
        model = load_model(self.keras_facenet)
        print('Keras Facenet Model Carregado')
        # Fazer Embedding De Cada Imagens De Treino Que Esta Convertida em Array numpy
        newTrainX = list()
        for face_pixels in trainX:
            embedding = self.get_embedding(model, face_pixels)
            newTrainX.append(embedding)
        newTrainX = asarray(newTrainX)
        # Fazer Embedding De Cada Imagens De Teste Que Esta Convertida em Array numpy
        newTestX = list()
        for face_pixels in testX:
            embedding = self.get_embedding(model, face_pixels)
            newTestX.append(embedding)
        newTestX = asarray(newTestX)
        # Comprimir As Imagens Embedded E Salvar Em Um Arquivo
        savez_compressed(self.faces_embeddings, newTrainX, trainy, newTestX, testy)
        return

    # Retornar A Predição De Cada Imagem Sob Face Embedding
    def get_embedding(self, model, face_pixels):
        # Escalar Valores De Pixel
        face_pixels = face_pixels.astype('float32')
        # Aplicar Standardization : Calculo de Média Aritmética / Desvio Padrão Nas Imagems Numéricas
        mean, std = face_pixels.mean(), face_pixels.std()
        face_pixels = (face_pixels - mean) / std
        # Expandir A Forma Da Imagem Que Esta Convertido Em Array
        samples = expand_dims(face_pixels, axis=0)
        # Fazer A Predição Sobre As Imagems Em Embedding
        yhat = model.predict(samples)
        return yhat[0]

    # Retornar A Predição Das Imagens De Treino e Teste Sob a Classificador svm-SVC
    def classifier(self):
        data = load(self.faces_embeddings)
        trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
        print('Dataset: treino=%d, teste=%d' % (trainX.shape[0], testX.shape[0]))
        # Normalizar Com l2 As Imagens Embedded
        in_encoder = Normalizer(norm='l2')
        trainX = in_encoder.transform(trainX)
        testX = in_encoder.transform(testX)
        # Aplicar o labelEncoder Noas Imagens De Treino E Teste
        out_encoder = LabelEncoder()
        out_encoder.fit(trainy)
        trainy = out_encoder.transform(trainy)
        testy = out_encoder.transform(testy)
        # Treinar Modelo svm-SVC Com As Imagens de Treino
        model = SVC(kernel='linear', probability=True)
        model.fit(trainX, trainy)
        # Salvar O Modelo Treinado Em Um Local No Computador
        filename = self.svm_classifier
        pickle.dump(model, open(filename, 'wb'))
        # Predizer As Imagens De Treino e Teste
        yhat_train = model.predict(trainX)
        yhat_test = model.predict(testX)
        # Regornar A Pontuação Da Precisão Da Predição Sobre As Imagens De Treino e Teste
        score_train = accuracy_score(trainy, yhat_train)
        score_test = accuracy_score(testy, yhat_test)
        print('Acurácia: treino=%.3f, teste=%.3f' % (score_train * 100, score_test * 100))
        return

    # Iniciar Processo Treinamento Do Algoritmo
    def start(self):
        start_time = time.time()
        st = datetime.datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')
        print("----------------------------------------------------------------------------------------")
        print("Analise Reconhecimento Facial Iniciado em {}".format(st))
        print("----------------------------------------------------------------------------------------")
        # Criar Arquivo Comprimido De Todas As Faces Convertidas Em Array, Que Serão Treinadas E Validadas
        self.create_faces_npz()
        # Retornar O Arquivo Criado Com A Predição Das Imagens De Treino E Teste Embedded
        self.create_faces_embedding_npz()
        # Retornar A Predição Das Imagens De Treino e Teste Sob a Classificador svm-SVC
        self.classifier()
        end_time = time.time()
        et = datetime.datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')
        print("----------------------------------------------------------------------------------------")
        print("Analise Reconhecimento Facial Finalizado em {}".format(et))
        print("Total Tempo Decorrido {} segs".format(round(end_time - start_time), 0))
        print("----------------------------------------------------------------------------------------")
        return

In [ ]:
import cv2
from PIL import Image
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
import pickle

# Identificar A Quem Pertence A Face Da Pessoa Através Da Webcam
class FaceDetector:
    def __init__(self):
        self.facenet_model = load_model("C:\\zproject-course\\deeplearn\\facenet\\facenet_keras.h5")
        self.svm_model = pickle.load(open("C:\\zproject-course\\deeplearn\\facerecognition\\SVM_classifier.sav", 'rb'))
        self.data = np.load('C:\\zproject-course\\deeplearn\\facerecognition\\faces_dataset_embeddings.npz')
        # Instanciar Objeto MTCNN Para Detectar Face
        self.detector = MTCNN()

    # Detectar Face Pela Webcam E Retornar Coordenadas Do Local Da Face
    def face_mtcnn_extractor(self, frame):
        result = self.detector.detect_faces(frame)
        return result

    # Trazer As Coordenadas Da Face Para Ser Identificada Pelo Classificador
    def face_localizer(self, person):
        bounding_box = person['box']
        x1, y1 = abs(bounding_box[0]), abs(bounding_box[1])
        width, height = bounding_box[2], bounding_box[3]
        x2, y2 = x1 + width, y1 + height
        return x1, y1, x2, y2, width, height

    # Reprocessar As Imagens Aplicando Standardization, Normalization, Embedding
    # Retornar A Coordenada Predita da Imagem Sob O modelo facenet_keras.h5
    def face_preprocessor(self, frame, x1, y1, x2, y2, required_size=(160, 160)):
        # Redimensiona A Imagem
        face = frame[y1:y2, x1:x2]
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = np.asarray(image)
        # Converte A Imagem Para Tipo Float32 Da numpy
        face_pixels = face_array.astype('float32')
        # Aplica O Standardization
        mean, std = face_pixels.mean(), face_pixels.std()
        face_pixels = (face_pixels - mean) / std
        # Expande A Imagem Array
        samples = np.expand_dims(face_pixels, axis=0)
        # Retorna A Coordenada Predita Da Imagem
        yhat = self.facenet_model.predict(samples)
        face_embedded = yhat[0]
        # Normalizar A Imagem-Vetores
        in_encoder = Normalizer(norm='l2')
        X = in_encoder.transform(face_embedded.reshape(1, -1))
        return X

    # Faz A Predição Sobre A Imagen Filmada Pela Webcam, Sob a Modelo Treinado pelo svm-SVC
    # Aplica O LabelEncoder e Retorna A Informação Predita E A Probabilidade Da Identificação
    def face_svm_classifier(self, X):
        # Predição Sob a Modelo Treinado pelo svm-SVC
        yhat = self.svm_model.predict(X)
        label = yhat[0]
        yhat_prob = self.svm_model.predict_proba(X)
        probability = round(yhat_prob[0][label], 2)
        trainy = self.data['arr_1']
        # Predição Sob Labelencoder
        out_encoder = LabelEncoder()
        out_encoder.fit(trainy)
        predicted_class_label = out_encoder.inverse_transform(yhat)
        label = predicted_class_label[0]
        return label, str(probability)

    # Retornar O Resultado Da Predição Em Tempo Real Na Webcam : Nome e Probabilidade
    def face_detector(self):
        # Inicializar Objeto Que Habilita Camera/WebCam Para Capturar Imagem. 0 = WebCam Notebook/PC
        cap = cv2.VideoCapture(0)
        # Enquanto A Webcam Estiver Ativa -Visualizando Imagem
        while True:
            # Detectar Face Em Cada Frame
            __, frame = cap.read()
            # Detectar Face Pela Webcam E Retornar Coordenadas Do Local Da Face
            result = self.face_mtcnn_extractor(frame)
            if result:
                # Desenhar Retângulo Na Face Detectada Utilizando As Coordenadas
                for person in result:
                    # Trazer As Coordenadas Da Face Para Ser Identificada Pelo Classificador
                    x1, y1, x2, y2, width, height = self.face_localizer(person)
                    # Reprocessar As Imagens Aplicando Standardization, Normalization, Embedding
                    # Retornar A Coordenada Predita da Imagem Sob O modelo facenet_keras.h5
                    X = self.face_preprocessor(frame, x1, y1, x2, y2, required_size=(160, 160))
                    # Predizer A Etiqueta Com Nome E Probabilidade
                    label, probability = self.face_svm_classifier(X)
                    print(" Person : {} , Probability : {}".format(label, probability))
                    # Desenhar Linhas Do Retangulo Sobre A Imagem
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 155, 255), 2)
                    # Exibir Na Janela WebCam O Nome Localizado E A Probabilidade
                    cv2.putText(frame, label+probability, (x1, height),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255),
                                lineType=cv2.LINE_AA)
            # Exibir A Janela Infinitamente Ate Uma Tecla Ser Pressionada Para Uma Ação Qualquer
            k = cv2.waitKey(1)
            # Mostrar Janela-Frame Com Descrição No Topo
            cv2.imshow("Identificando Imagem - ESC = Finalizar", frame)
            # Verificar Se Foi Pressionado A Tecla 27=ESC Para Fechar Webcam
            if k%256 == 27:
                k = cv2.waitKey(1)
                print("")
                print("ESC pressionado. Fechando Câmera")
                break

        # Ao Final Da Uso da Webcam Limpa As Variáveis Objeto Webcam E
        # Janelas Utilizadas Neste Processo, Respectivamente
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
import os
import subprocess
import cv2
from mtcnn.mtcnn import MTCNN

# Classe 
class MainMenu:
    def __init__(self):
        self.detector = MTCNN()
        print("= MACHINE LEARNING NO SISTEMA RECONHECIMENTO FACIAL ATRAVÉS DE CAMERA - OpenVC, MTCNN, LabelEncoder =")

    def homemenu(self):
        print("")
        print("Digite O Número Relacionado a Opção Desejada")
        print("")
        print("1 Gravar Imagem")
        print("2 Analisar Imagem")
        print("3 Identificar Imagem")
        print("0 Sair Do Sistema")
        choice = 0
        try:
            choice = int(input(" >> "))
        except ValueError:
            print("")
            print("== Atenção ==")
            print("Digite Valor Númerico")
            print("             =======")
            self.homemenu()

        self.choice = choice
        self.callactions(str(self.choice))
   
    def saveperson(self):
        os.system('cls')
        print("Gravando...")
        print("Digitar Dados Do Participante")
        gravar = RegisterImg()
        gravar.saveperson()

        os.system('cls')
        self.homemenu()
        return

    def analyze(self):
        print("Analisando Imagem ...")
        analisar = FaceTrainer()
        analisar.start()
        os.system('cls')
        self.homemenu()
        return

    def identifyimg(self):
        print("Identificando Imagem ...")
        identificar = FaceDetector()
        identificar.face_detector()
        os.system('cls')
        self.homemenu()
        return

    def exitsystem(self):
        print("Sistema Encerrado !")
        os.system('cls')
        quit()
        return

    def callactions(self,ch):
        self.actionsmenu(ch)
        
        return

    def actionsmenu(self,opcao):
        menu_actions = {
                        'main_menu': self.homemenu,
                        '1': self.saveperson,
                        '2': self.analyze,
                        '3': self.identifyimg,
                        '0': self.exitsystem
                        }
        menu_actions[opcao]()
        return

In [ ]:
# Chamar Classe MainMenu Para Exibir Menu Que Será Manipulado
if __name__ == "__main__":
    os.system('cls')
    menu = MainMenu()
    menu.homemenu()